In [40]:
from keras.datasets import mnist 
import numpy as np

#*Importing the dataset

In [41]:
(train_images , train_labels),(test_images , test_labels) = mnist.load_data() 

In [42]:
print(train_images.shape , train_labels.shape)
print(test_images.shape , test_labels.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


#*subset the data to use only class 0 and class1

In [43]:
train_indexes = (train_labels==0)|(train_labels==1)
test_indexes = (test_labels==0)|(test_labels==1)
x_train = train_images[train_indexes]
y_train = train_labels[train_indexes]

x_test = test_images[test_indexes]
y_test = test_labels[test_indexes]

#reshape training data
x_train = x_train.reshape(-1 , 784)
y_train = y_train.reshape(-1 , 1)

x_test = x_test.reshape(-1 , 784)
y_test = y_test.reshape(-1 , 1)


#*standardization of the data

In [44]:
mean_x_train = np.mean(x_train)
std_x_train = np.std(x_train)
x_train = (x_train - mean_x_train)/std_x_train


mean_x_test = np.mean(x_test)
std_x_test = np.std(x_test)
x_test = (x_test - mean_x_test)/std_x_test

#*Logistic Regression

In [45]:
def logistic_regression(X, y, lr, num_iterations):
   
    m, n = X.shape
    w = np.zeros((n, 1))
    b = 0
    limit = 10 ** -8

   
    for i in range(num_iterations):
        
        z = np.dot(X, w) + b
        A = 1 / (1 + np.exp(-z))
        
        # cost function
        error = (-1/m) * np.sum(y*np.log(A) + (1-y)*np.log(1-A))
        
        
        dw = np.mean( np.dot(X.T, (A-y)))
        db = np.mean( np.sum(A-y))
        
        # Update parameters
        w = w - lr * dw
        b = b - lr * db
        if (error < limit):
          break
        
    return w, b

#*Divide data into training and validation set using 10-fold cross validation method

In [46]:
num_folds = 10
learning_rates = [0.001, 0.01, 0.1, 1]
accuracies = []
# Shuffle the data and labels
idx = np.arange(x_train.shape[0])
np.random.shuffle(idx)
x_train = x_train[idx]
y_train = y_train[idx]

# Split the data and labels into num_folds folds
data_folds = np.array_split(x_train, num_folds)
labels_folds = np.array_split(y_train, num_folds)

for i in range(num_folds):
    # Select the i-th fold as the validation set
    val_data = data_folds[i]
    val_labels = labels_folds[i]
    
    # Use the remaining folds as training data
    train_data_fold = np.concatenate(data_folds[:i] + data_folds[i+1:])
    train_labels_fold = np.concatenate(labels_folds[:i] + labels_folds[i+1:])

In [49]:
# Implement Logistic Regression with different values for learning rate
    
    

for lr in learning_rates:
      w, b = logistic_regression(train_data_fold , train_labels_fold.reshape(-1,1), lr, num_iterations=1000)
        
      # Predict on validation set
      z = np.dot(val_data, w) + b
      A = 1 / (1 + np.exp(-z))
      predictions = (A > 0.5).astype(int)
      acc = np.mean(predictions == val_labels .reshape(-1,1))
      accuracies.append(acc)
      print('Learning rate: {}, Accuracy: {:.2f}'.format(lr, acc))



<ipython-input-45-f70b91815a07>:15: RuntimeWarning: divide by zero encountered in log
  error = (-1/m) * np.sum(y*np.log(A) + (1-y)*np.log(1-A))
<ipython-input-45-f70b91815a07>:15: RuntimeWarning: invalid value encountered in multiply
  error = (-1/m) * np.sum(y*np.log(A) + (1-y)*np.log(1-A))


Learning rate: 0.001, Accuracy: 0.93


<ipython-input-45-f70b91815a07>:12: RuntimeWarning: overflow encountered in exp
  A = 1 / (1 + np.exp(-z))


Learning rate: 0.01, Accuracy: 0.94


<ipython-input-49-33c7fb3f4f3c>:10: RuntimeWarning: overflow encountered in exp
  A = 1 / (1 + np.exp(-z))


Learning rate: 0.1, Accuracy: 0.93
Learning rate: 1, Accuracy: 0.94


In [50]:
# Report difference accuracy for the different learning rate
acc_diff = np.array(accuracies).reshape(-1, len(learning_rates)).mean(axis=0)
for i, lr in enumerate(learning_rates):
    print('Learning rate: {}, Average accuracy: {:.2f}'.format(lr, acc_diff[i]))

Learning rate: 0.001, Average accuracy: 0.93
Learning rate: 0.01, Average accuracy: 0.94
Learning rate: 0.1, Average accuracy: 0.93
Learning rate: 1, Average accuracy: 0.94
